In [58]:
from PCBDataset import PCBDataset
from os import path

root = r"C:\Projects\UKM\PCB_DATASET"
info_path = path.join(root, "Annotations")
image_path = path.join(root, "images")
image_src = path.join(root, "PCB_USED")
dataset = PCBDataset()
# dataset = PCBDataset("dataset.pth")
df = dataset.parse(info_path)

In [59]:
import numpy

def get_image_file(image_id: str):
    ids = image_id.split('_')[1:-1]
    ids[0] = ids[0].title()
    class_path = '_'.join(ids)
    image_file = path.join(image_path, class_path, image_id +".jpg")
    return image_file

def get_src_file(image_id: str):
    id = image_id.split('_')[0]
    image_file = path.join(image_src, f"{id}.jpg")
    return image_file


In [60]:
from PIL import Image
import numpy
from os import walk

groups = df["file"].unique()
classes = df["class"].unique()
boxes = {}
for _, _, files in walk(image_src):
    for file in files:
        id = file.split('.')[0]
        boxes[id] = {}
        for c in classes:
            boxes[id][c] = []

for group in groups:
    ids = group.split('_')
    id = ids[0]
    c = '_'.join(ids[1:-1])
    rows = df[df['file'] == group]
    for i,row in rows.iterrows():
        boxes[id][c].append(row)

for id, boxC in boxes.items():
    print(f"\n{id}")
    for c, box in boxC.items():
        print(f"{c}{''.rjust(20-len(c))}: {len(box)}")


01
missing_hole        : 61
mouse_bite          : 53
open_circuit        : 59
short               : 72
spur                : 53
spurious_copper     : 60

04
missing_hole        : 60
mouse_bite          : 60
open_circuit        : 59
short               : 59
spur                : 60
spurious_copper     : 60

05
missing_hole        : 46
mouse_bite          : 51
open_circuit        : 32
short               : 30
spur                : 50
spurious_copper     : 50

06
missing_hole        : 51
mouse_bite          : 52
open_circuit        : 50
short               : 55
spur                : 49
spurious_copper     : 50

07
missing_hole        : 50
mouse_bite          : 50
open_circuit        : 50
short               : 50
spur                : 50
spurious_copper     : 50

08
missing_hole        : 49
mouse_bite          : 50
open_circuit        : 50
short               : 49
spur                : 50
spurious_copper     : 50

09
missing_hole        : 53
mouse_bite          : 50
open_circuit        : 

In [63]:
def patch_image(img_src: numpy.array, img_id: str, img_class: str, box_id: int = -1):
    boxList = boxes[img_id][img_class]
    names = img_class.split('_')
    names[0] = names[0].title()
    class_path = '_'.join(names)
    if box_id < 0:
        box_id = numpy.random.randint(len(boxList))
    box = boxList[box_id]
    image_file = path.join(image_path, class_path, box['file'] + ".jpg")
    src_path = path.join(image_src, f"{img_id}.jpg")
    img = numpy.array(Image.open(image_file))
    img_src[box['ymin']:box['ymax'], box['xmin']:box['xmax'],:] = img[box['ymin']:box['ymax'], box['xmin']:box['xmax'],:]
    # print(img_id, img_class, box_id, box['file'],box['xmin'], box['ymin'])
    # return Image.fromarray(img_src)
    return box_id

def generate_image(img_id: str, min = 1, max = 6):
    size = numpy.random.randint(min, max)
    cc = numpy.random.randint(len(classes), size=size)
    img_src = numpy.array(Image.open(path.join(image_src, f"{img_id}.jpg")))
    names = []
    for ci in cc:
        c = classes[ci]
        b = patch_image(img_src, img_id, c)
        names.append(f'{ci}-{b}')
    dst = f"{img_id}_{'_'.join(names)}.jpg"
    Image.fromarray(img_src).save(dst)
    print(dst)

generate_image("01")

01_4-9_0-50.jpg


In [13]:
#torch.save((dataset.images, dataset.labels), 'dataset.pth')

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import time
from pathlib import Path

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") # device object
batch_size = 16
size = 128
tx = transforms.Compose([
    transforms.Resize([size, size]),
    transforms.ToTensor(),
])
# root = r"C:\Projects\UKM\PCB_DATASET"
# dataset = PCBDataset(root, tx)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

model = ResNetFCM()#.to(device)

def lossFn(no, outputs, nl, labels):
    loss_ln = F.cross_entropy(no, nl)
    vlabels = labels.permute(1, 0)
    loss = 0
    n = len(outputs)
    for i in range(n):
        loss += F.cross_entropy(outputs[i], vlabels[i])
    return loss_ln + loss / n

def accMC(no, outputs, nl, labels):
    n = labels.shape[0]
    _, np = torch.max(no, 1)
    accL = sum(nl == np)
    o = torch.IntTensor(labels.shape).to(device)
    for i in range(len(outputs)):
        _,preds = torch.max(outputs[i], 1)    
        o[:,i] = preds
    acc = 0
    for i, label in enumerate(labels):
        v = sum(o[i] == label).item()
        acc += v / 5
        # if nl[i] == 1:
        #     v = sum(o[i] == label[0]).item() if nl[i] == np[i] else 0
        #     #print('single', nl[i].item(), np[i].item(), v)
        #     acc += 1 if v > 0 else 0
        # else:
        #     v = sum(o[i] == label).item()
        #     acc += v / 5
        #     #print('multi', nl[i], v)
        #print(label)
        #print(o[i])
    return accL.item(), acc

def train():
    #model.load_state_dict(torch.load(model_path))
    criterion = nn.CrossEntropyLoss()  #(set loss function)
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    num_epochs = 20   #(set no of epochs)
    start_time = time.time() #(for showing time)
    max_acc = 0
    for epoch in range(num_epochs): #(loop for every epoch)
        model.train()    #(training model)
        running_loss = 0.   #(set loss 0)
        # load a batch data of images
        accLs = 0
        accCs = 0
        for i, (inputs, labels, nl) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device) 
            nl = nl.to(device)
            # forward inputs and get output
            optimizer.zero_grad()
            no, outputs = model(inputs)
            # get loss value and update the network weights
            loss = lossFn(no, outputs, nl, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            # accuracy
            accL, accC = accMC(no, outputs, nl, labels)
            accLs += accL
            accCs += accC
        
        test_accL, test_accC = test()
            
        epoch_loss = running_loss / len(train_dataset)
        train_accL = accLs / len(train_dataset) * 100.
        train_accC = accCs / len(train_dataset) * 100.
        
        
        print(f'Epoch [{epoch + 1:04d} / {num_epochs}] Loss: {epoch_loss:.4f} Train Acc Length: {train_accL:.4f}% Train Acc Class: {train_accC:.4f}% Test AccL: {test_accL:.4f}% Test AccC: {test_accC:.4f}% Time: {(time.time() -start_time):.4f}s')    
        if max_acc < test_accC:
            max_acc = test_accC
            print(f"saving model {model_path}")
            torch.save(model.state_dict(), model_path)

def test():
    """ Testing Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        accLs = 0
        accCs = 0
        for inputs, labels, nl in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            nl = nl.to(device)
            no, outputs = model(inputs)
            #loss = lossFn(no, outputs, nl, labels)
            #running_loss += loss.item() * inputs.size(0)
            accL, accC = accMC(no, outputs, nl, labels)
            accLs += accL
            accCs += accC
            
        #epoch_loss = running_loss / len(test_dataset)
        test_accL = accLs / len(test_dataset) * 100.
        test_accC = accCs / len(test_dataset) * 100.
    return test_accL, test_accC

# train()
# model.load_state_dict(torch.load(model_path))
# test_accL, test_accC = test()
# print(f"length accuracy {test_accL:.4f} class accuracy {test_accC:.4f}")

693


In [60]:
from  matplotlib import image, patches, pyplot
from pandas.core.groupby.generic import DataFrameGroupBy

def plot_image(image_name: str, df_grp: DataFrameGroupBy, image_path: str):
    image_group = df_grp.get_group(image_name)    
    bbox = image_group.loc[:,['xmin', 'ymin', 'xmax', 'ymax']]
    class_name = image_group['class'].values[0]
    class_names = class_name.split('_')
    class_names[0] = class_names[0].title()
    class_path = '_'.join(class_names)
    filename = path.join(image_path, class_path, f"{image_name}.jpg")
    img = image.imread(filename)
    _,ax = pyplot.subplots(figsize=(18,10))
    ax.imshow(img,cmap='binary')
    for i in range(len(bbox)):
        box = bbox.iloc[i].values
        x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none',)
        ax.text(*box[:2], class_name, verticalalignment='bottom', color='white', fontsize=13, weight='bold')
        ax.add_patch(rect)
    pyplot.show()

plot_image('12_spur_01', df_grp, r'C:\Projects\UKM\PCB_DATASET\images')

: 

: 